Data Preparation: Missing values, feature engineering, encoding, train-test split.
Model Building: Linear Regression, Random Forest, XGBoost for both Regression (Claim Severity) and Classification (Claim Probability).
Model Evaluation: RMSE, R-squared for regression; Accuracy, Precision, Recall, F1-score for classification.
Model Interpretability: SHAP/LIME.
Modular Code: Putting models and helper functions into models.py.

In [4]:
# notebooks/03_model_development.ipynb


# 1. Setup and Data Loading
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBRegressor, XGBClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import shap

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Initialize df to None outside the try block
df = None

# Load the processed data
# Assuming cleaned_data.parquet is saved in the 'Data/processed/' directory
try:
    # --- CHANGE THIS LINE ---
    df = pd.read_parquet('../Data/processed/cleaned_data.parquet') # <--- CHANGE THIS FILENAME
    # --- TO THIS ONE ---
    print("DataFrame loaded successfully.")
except FileNotFoundError:
    print("Error: 'Data/processed/cleaned_data.parquet' not found.")
    print("Please ensure the processed data from Task 2/3 is saved in 'Data/processed/cleaned_data.parquet'.")
    print("Exiting as core data is missing.")
    raise FileNotFoundError("Processed DataFrame 'cleaned_data.parquet' not found at the specified path.")
except Exception as e:
    print(f"An unexpected error occurred while loading the DataFrame: {e}")
    raise

# --- ONLY PROCEED IF DF IS LOADED ---
if df is not None:
    # Display basic info
    print("\n--- DataFrame Info ---")
    df.info()

    print("\n--- Missing Values ---")
    print(df.isnull().sum())

    print("\n--- Value Counts for Key Categorical Candidates ---")
    # Identify potential categorical columns. Based on typical insurance data, and assuming some initial processing.
    # 'PolicyID', 'ClaimID', 'TotalClaims' (regression target) should not be treated as categorical features.
    # We will derive 'HasClaim' as a new binary column.
    categorical_cols_initial_check = ['PolicyType', 'VehicleUse', 'CoverageType', 'Province', 'Gender']

    for col in categorical_cols_initial_check:
        if col in df.columns:
            print(f"\nValue Counts for {col}:")
            print(df[col].value_counts())
        else:
            print(f"Warning: Column '{col}' not found in DataFrame.")

    # Check numerical features that might be discrete or low-cardinality and could be treated as categorical
    numerical_categorical_candidates = ['VehicleYear']
    for col in numerical_categorical_candidates:
        if col in df.columns and df[col].nunique() < 20:
            print(f"\nUnique values for numerical candidate {col}:")
            print(df[col].value_counts().sort_index())
    print("\nInitial data loading and inspection complete. Proceed to next cells.")

else:
    print("DataFrame 'df' was not loaded. Subsequent operations will fail.")

DataFrame loaded successfully.

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000098 entries, 0 to 1000097
Data columns (total 53 columns):
 #   Column                       Non-Null Count    Dtype         
---  ------                       --------------    -----         
 0   underwritten_cover_id        1000098 non-null  int64         
 1   policy_id                    1000098 non-null  int64         
 2   transaction_month            1000098 non-null  datetime64[ns]
 3   is_vat_registered            1000098 non-null  bool          
 4   citizenship                  1000098 non-null  object        
 5   legaltype                    1000098 non-null  object        
 6   title                        1000098 non-null  object        
 7   language                     1000098 non-null  object        
 8   bank                         854137 non-null   object        
 9   account_type                 959866 non-null   object        
 10  marital_status         

In [5]:
# 03_model_development.ipynb (Cont.)

# 2. Data Preparation

# Define the two modeling datasets
# Target variable for probability model: 1 if claim, 0 if no claim
# Create this new column 'has_claim'
df['has_claim'] = df['total_claims'].apply(lambda x: 1 if x > 0 else 0)

# Dataset for Claim Severity Prediction (Regression): Policies with claims
df_severity = df[df['has_claim'] == 1].copy()
# Target variable for severity model
y_severity = df_severity['total_claims']


# Feature Engineering (Apply to df, then create X_severity and X_probability)
def feature_engineer(dataframe):
    df_fe = dataframe.copy()

    # --- Date Features ---
    # Convert 'transaction_month' to datetime if not already (info says datetime64[ns], but good to be explicit)
    df_fe['transaction_month'] = pd.to_datetime(df_fe['transaction_month'])
    df_fe['month'] = df_fe['transaction_month'].dt.month
    df_fe['year'] = df_fe['transaction_month'].dt.year
    df_fe['day_of_week'] = df_fe['transaction_month'].dt.dayofweek # Monday=0, Sunday=6

    # Vehicle Age at transaction (assuming 'registration_year' is vehicle's manufacturing/registration year)
    # Be careful with 'registration_year' vs. 'vehicle_intro_date'
    # Let's use 'registration_year' as 'vehicle_intro_date' has missing values and is object dtype.
    df_fe['vehicle_age_at_transaction'] = df_fe['year'] - df_fe['registration_year']
    # Handle potential negative or zero ages for new vehicles if registration year is same as transaction year
    df_fe['vehicle_age_at_transaction'] = df_fe['vehicle_age_at_transaction'].apply(lambda x: max(0, x))


    # Interaction features
    df_fe['exposure_x_kilowatts'] = df_fe['total_premium'] / df_fe['calculated_premium_per_term'] * df_fe['kilowatts'] # Approximation of Exposure
    df_fe['claims_per_kilowatt'] = df_fe['total_claims'] / (df_fe['kilowatts'] + 1e-6) # Add small constant to avoid div by zero

    # Binning for a continuous variable if desired (e.g., cubic_capacity)
    # For now, keeping as continuous, but remember this option for further refinement.


    # Handle specific 'object' columns that might be boolean-like or numeric that were ingested as object
    # If `new_vehicle`, `written_off`, etc., are strings like 'Yes'/'No', convert to 1/0
    for col_bool in ['new_vehicle', 'written_off', 'rebuilt_vehicle', 'converted_vehicle', 'alarm_immobiliser', 'tracking_device', 'cross_border', 'is_vat_registered']:
        if col_bool in df_fe.columns:
            # First, fill NaN or a placeholder, then map
            # Assuming 'Yes'/'True' maps to 1, others to 0 or specific handling for NaN
            df_fe[col_bool] = df_fe[col_bool].map({'Yes': 1, 'No': 0, True: 1, False: 0}).fillna(-1).astype(int) # -1 for missing or unknown

    return df_fe

df_fe = feature_engineer(df)

# --- Missing Data Handling ---
# Columns with 100% missing values should be dropped
cols_to_drop_due_to_missing = ['number_of_vehicles_in_fleet', 'term_frequency']
df_fe = df_fe.drop(columns=cols_to_drop_due_to_missing, errors='ignore')

# Impute 'bank', 'account_type', 'new_vehicle' (if not handled by mapping above) with a placeholder 'Missing'
for col in ['bank', 'account_type']: # 'new_vehicle' should be handled by mapping now
    if col in df_fe.columns:
        df_fe[col] = df_fe[col].fillna('Missing')

# Impute numerical columns with low missing values (mm_code, cylinders, number_of_doors, vehicle_intro_date)
# For 'vehicle_intro_date', convert to numeric (e.g., year) if possible, or drop if too complex for simple imputation.
# Given it's an object, let's just drop it for now for simplicity, or convert it to a meaningful numerical feature like 'intro_year'.
# If 'vehicle_intro_date' is like 'YYYY-MM-DD', we can extract year and fill NaNs.
if 'vehicle_intro_date' in df_fe.columns:
    # Try converting to datetime and extracting year, then impute
    df_fe['vehicle_intro_year'] = pd.to_datetime(df_fe['vehicle_intro_date'], errors='coerce').dt.year
    df_fe['vehicle_intro_year'] = df_fe['vehicle_intro_year'].fillna(df_fe['vehicle_intro_year'].median()) # Impute with median year
    df_fe = df_fe.drop(columns=['vehicle_intro_date'], errors='ignore') # Drop original object column

# For other numerical columns, impute with median
for col in ['mm_code', 'cylinders', 'number_of_doors']:
    if col in df_fe.columns and df_fe[col].isnull().any():
        df_fe[col] = df_fe[col].fillna(df_fe[col].median())


# Define features for modeling
# Drop original target variables, IDs, and columns already handled or with too many unique values for OHE
# 'postal_code', 'make', 'model' can have very high cardinality. For now, dropping 'postal_code', 'make', 'model'.
# In a real project, 'make'/'model' might need more advanced embedding or grouping.
# 'title', 'language', 'country', 'main_crestazone', 'sub_crestazone', 'itemtype', 'product', 'section', 'statutory_class', 'statutory_risk_type'
# These are still many categorical features. We might need to reduce the list for simpler models, or expect OHE to create many columns.

features_to_drop_overall = [
    'underwritten_cover_id', 'policy_id', 'transaction_month',
    'total_claims', 'has_claim', # These are targets or derived targets
    'calculated_premium_per_term', # This is a conceptual target for optimization, not a feature for severity/probability
    'loss_ratio', # Derived from claims and premium, can cause data leakage
    'postal_code', 'make', 'model' # High cardinality, dropping for simplicity
]

# Ensure we're only dropping columns that exist in the DataFrame
features_to_drop_overall = [f for f in features_to_drop_overall if f in df_fe.columns]

X_all = df_fe.drop(columns=features_to_drop_overall, errors='ignore')

# Create X for severity model (only policies with claims)
X_severity = X_all[df_fe['has_claim'] == 1].copy()
# Ensure y_severity is aligned with X_severity's index
y_severity = df_fe.loc[X_severity.index, 'total_claims']


# Create X for probability model (all policies)
X_probability = X_all.copy()
y_probability = df_fe['has_claim']


# Identify numerical and categorical columns for preprocessing pipelines AFTER dropping/engineering
numerical_cols = X_probability.select_dtypes(include=np.number).columns.tolist()
# Filter out boolean columns if they were not converted to int during feature engineering
numerical_cols = [col for col in numerical_cols if col not in ['is_vat_registered_bool', 'new_vehicle', 'written_off', 'rebuilt_vehicle', 'converted_vehicle', 'alarm_immobiliser', 'tracking_device', 'cross_border'] ] # Assuming these are mapped to int now

categorical_cols = X_probability.select_dtypes(include='object').columns.tolist()
# Add the boolean/integer-mapped columns that might still be considered categorical by the model if they are 0/1/-1
# (e.g., for tree-based models, they work fine as numbers, but for linear models, they might need OHE depending on how we treat -1)
# For simplicity, let's keep boolean mapped values as numerical for now and let the scaler handle them.

# Final check for numerical/categorical lists
print("\n--- Columns after Feature Engineering and Missing Value Handling ---")
print("Total Features:", len(X_probability.columns))
print("Numerical Features identified:", len(numerical_cols), numerical_cols)
print("Categorical Features identified:", len(categorical_cols), categorical_cols)


# Create preprocessing pipelines
# Numerical: Standard Scaling
# Categorical: One-Hot Encoding (handle_unknown='ignore' for unseen categories in test set)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough' # Keep other columns if any (e.g. if some columns are bool dtype, they will pass through)
)


# Train-Test Split for both models
# Severity Model (only for policies with claims)
X_train_severity, X_test_severity, y_train_severity, y_test_severity = train_test_split(
    X_severity, y_severity, test_size=0.2, random_state=42
)
print(f"\nSeverity Model - Training Data Shape: {X_train_severity.shape}, Test Data Shape: {X_test_severity.shape}")

# Probability Model (all policies)
X_train_probability, X_test_probability, y_train_probability, y_test_probability = train_test_split(
    X_probability, y_probability, test_size=0.2, random_state=42, stratify=y_probability # Stratify for imbalanced classes
)
print(f"Probability Model - Training Data Shape: {X_train_probability.shape}, Test Data Shape: {X_test_probability.shape}")
print(f"Probability Model - Train Claim Ratio: {y_train_probability.mean():.4f}")
print(f"Probability Model - Test Claim Ratio: {y_test_probability.mean():.4f}")


--- Columns after Feature Engineering and Missing Value Handling ---
Total Features: 48
Numerical Features identified: 18 ['is_vat_registered', 'mm_code', 'registration_year', 'cylinders', 'cubic_capacity', 'kilowatts', 'number_of_doors', 'custom_value_estimate', 'capital_outstanding', 'sum_insured', 'total_premium', 'month', 'year', 'day_of_week', 'vehicle_age_at_transaction', 'exposure_x_kilowatts', 'claims_per_kilowatt', 'vehicle_intro_year']
Categorical Features identified: 23 ['citizenship', 'legaltype', 'title', 'language', 'bank', 'account_type', 'marital_status', 'gender', 'country', 'province', 'main_crestazone', 'sub_crestazone', 'itemtype', 'vehicle_type', 'body_type', 'excess_selected', 'cover_category', 'cover_type', 'cover_group', 'section', 'product', 'statutory_class', 'statutory_risk_type']

Severity Model - Training Data Shape: (2230, 48), Test Data Shape: (558, 48)
Probability Model - Training Data Shape: (800078, 48), Test Data Shape: (200020, 48)
Probability Model